## Board Class
### Overview
This class contains the entire functionality of checkers. It handles everything required to turn the game of checkers into a trainable game for the different AI algorithms. The functions that we have learned throughout the semester that are quintessential to many algorithms, such as `stateMoveTuple`, `validMoves`, etc. all reside in the board class. Since the class is very large, the functions will be described in the order they appear to make it easier to understand. 

### Functions

#### Constructor: `__init__(self)`
This function initializes the game of checkers. It uses an 8x8 numpy array called `draught` (the official name of the checkers board) of `Piece` objects to initialize the board. If a piece is absent, the space is set to `None`. Initial positions are the first three and last three rows and alternate between a left alignment and right alignment. Since the color red starts the game, the turn is set to `Color.RED`, where color is an enumeration. This could have been simplified to a boolean saying `red_turn = True` for red's turn and `red_turn = False` for black's turn. However for readability of my teammates I added the enumeration.

#### Representation: `__repr__(self)`
This function converts a board to a string so it is easier to print and make `stateMoveTuples`. It creates a format string that represents a row, and for every row add the `Piece` string representation and if it is `None` add a dash to represent the empty space. It has indexed columns and rows that correspond to the indices of the numpy array `draught`.

#### Print State: `printState(self)`
This function prints a board, which calls the `__repr__` function described earlier.

#### State Move Tuple: `stateMoveTuple(self, move)`
This function takes a move as input and returns a tuple with the state of the board as a string and a move (which is already a tuple).

#### Make Move: `makeMove(self, move)`
This function takes a move and will make a move on the board. It uses the length of the move tuple to determine if any jumps occur during the move. If a jump is made, the pieces that were jumped during the move are set to `None` in the `draught`. It will then set the pieces previous space to `None` and update the space that it is moving to with moved `Piece`. If a `Piece` has reached the opponents edge of the `draught`, the `Piece` will be promoted to a king. The turn is then changed.

#### Change Turn: `changeTurn(self)`
This changes the turn of the board using a ternary operator to determine which turn to set the `Board` to the correct turn.

#### Valid Moves: `validMoves(self)`
This function determines the list of valid moves for a `Board` based on the turn. It begins by filtering for the `valid_pieces` on the `Board`. An array is initialized to hold all `valid_moves`. For every piece in the `valid_pieces` it checks if it can move left or right based on its color if it is not a king. So black can move up in rows and red can move down. Kings can move in all directions. This uses the `checkSpace` helper function to determine legal moves in that direction. It filters out the moves returned and adds it to the array it returns in the tuple format of `(initial position, move)`. For multiple jumps, they are recorded in the move. 

#### Check Space: `checkSpace(self, position, row_mod, col_mod, king=False, recurse=False, positions=None)`
This function takes as input an initial position, directions to move, whether or not the piece is a king, and recursion arguments. It starts by getting the `new_row` and `new_col` for a space to check. If it is in bounds, a check is performed to see if it is unoccupied. If it is, and it's not recursing (which happens during a jump), it will return that position only since it is an unoccupied space. If it is occupied and can be jumped and hasn't already been added to a jump pattern, it will check for extra jumps by checking the left and right positions again recursively. For kings it checks three positions instead of four (in `validMoves` it uses four) because a piece may not be jumped twice. If more jumps are found, they are added to the move. Return the list of valid moves.

#### Can Be Jumped: `canBeJumped(self, attacker, defender)`
Checks if a piece can be jumped. It takes as input an attacking position and a defending position. The massive if statement determines the direction of the attack and if the piece has no piece on the other side of a jump, it may be jumped and will return `True`. In the case that it cannot be jumped, it will return `False`. 

# Piece Class
## Overview
This class represents a `Piece` on the `Board` in the `Board.draught` array. The class is fairly simple, but contains data critical to a `Piece`'s functionality in the game.

## Functions

#### Constructor: `__init__(self, color, initial_position)`
The constructor take as input the `Color` of the `Piece`, and it's `initial_position` on the board, which is a tuple in the form of (row, column). It is initialized as a normal piece rather than a king, so `king = False`. 

#### Representation: `__repr__(self)`
This is the string function for a `Piece`. It is the `Color` of the piece as a character ('B' for `Color.BLACK` and 'R' for `Color.RED`. If it is a king, a 'K' is appended to the string. So a red king is represented as 'RK' instead of just 'R'.

#### King Me: `king_me(self)`
Promotes a `Piece` to king by setting `Piece.king` to `True`. 

# Reinforcement 
## Overview
Notice that this is not a class like `Board` and `Piece` but rather just functions. These functions will both train and use a `Q` dictionary that stores the a `Board` state move tuple as the key to a number that represents the potential game outcome of making that particular move for that exact scenario. The higher the number, the better that move is for the AI. Our reinforcement is a little bit different than the reinforcement learning we saw in class. This algorithm back propagates a reinforcement throughout every move that led to that state, so that moves early in the game still recieve a value. Also, we significantly reduce the size of the `Q` table by storing just moves, rather than a state move tuple. Checkers has 500 billion billion possible states, not including state move tuples. We aslo did this because moves hold interesting information about the game, such as jumping another piece, getting kinged, etc. due to the way they are represented in the board class, and so that AI can extrapolate the usefulness of the move based on the outcome of the game.

## Functions

#### Epsilon Greedy: `epsilonGreedy(epsilon, Q, board)`
This function selects a move for a board based on an epsilon to add randomness into the selection. It takes an `epsilon` and will choose either a random move if a randomly generated number is less than the `epsilon`. The process is that a list of valid moves is generated for a `Board`. If the random route is selected, the a random move from `valid_moves` is selected. Otherwise, the greedy move for the `Q` dictionary is selected and returned.

#### Greedy: `greedy(valid_moves, Q, board)`
This function will make the greedy move for a board, or basically the move with the most utility as determined by the `Q` dictionary. For every move in `valid_moves`, if the state move tuple is in the dictionary, get its determined utility. Otherwise, it will be 0, a neautral utility because its usefulness has not yet been determined. It will then return the move with the highest utility. If all moves have the same value (most likely zero at the beginning), the first valid move will be chosen. For red this is the first move for the piece with the lowest row first, and lowest column second.

#### Finished: `finished(board)`
Determines if a game is finished. If the length of valid moves from the `board` is zero, the game is over.

#### Train Q: `trainQ(nRepititions, learningRate, epsilonDecayFactor, propagationDecayFactor)`
This function trains a `Q` dictionary for the number `nRepititions` games. The `learningRate` determines how fast the `Q` dictionary will learn for a certain move and board state. The `epsilonDecayFactor` determines the rate at which the `epsilon` passed to `epsilonGreedy` will decay, meaning using the utilities stored in the `Q` more and more. Every game is a new game, in which the AI is set to play as red. For every game, if the game is finished, update the dictionary with a reinforcement. If it is a win, that move will be represented by a 1. If it is a loss it will be -2. We selected -2 since it is playing a random opponent, and losing to a random move maker should be punished harsher than winning against that random player. After a move has been made, it will back propagate that move's utility throughout every move made that led up to it by calling `backPropagateReinforcement`.

#### Back Propagate Reinforcement: `backPropagateReinforcement(Q, move_tuple_list, learningRate, propagationDecayFactor)`
This function will back propagate a reinforcement through the Q dictionary. The reason we added this is there were too many zeros in the Q dictionary, and thus only first moves were selected a decent amount of the time as a result of np.argmax returning the first valid move when a move doesn't have a reinforcement. It back propagates a reinforcement throughout every move made that led to the final board state, so that every move has a reinforcement (every move matters!).

#### Use Q: `useQ(Q, maxSteps)`
This function tests the `Q` dictionary that results from the `trainQ` function. It uses the same game logic as `trainQ`. Red starts the game (red is the reinforcement trained AI). It makes a greedy move by calling `greedy` to determine the optimal move. Then black makes a random move and the game continues until one has lost. 

## Observations
There were some interesting observations along the way that led to structuring our reinforcement AI the way we did. First of all, we saw that using state move tuples worked just fine and the AI would beat the random with state move tuples as the keys to the `Q` dictionary. However, upon inspection there were way too many states and `Q` was very sparse after training, with often only values for less than 10% of all entries. 

The first method we tried to solve this was to back propagate the reinforcement with a very low decay rate so that the reinforcement would reach the opening move. However, these states were never used because there were too many to ever get a hit on the table. So how was that AI winning? Well, `numpy.argmax` in the `greedy` function takes the first valid move, so the strategy was inadvertantly to move the first piece possible up and to the left if it could, and never to move the last row. Since black could never move into the last row and make a king, it runs out of valid moves and the AI would win. 

To try and reduce this strategy and allow `Q` to actually contain some semblance of a strategy, we reduced `Q` to only contain the move selected. The move has inherent state involved, such as where it is moving from, the spaces it will jump, and the end position. This seemed to be enough to allow the AI to still ~98-99% of the time against a random opponent. This way we could look at the Q dictionary and see the values for the moves it selected. It still used the strategy of avoiding moving out of the back row if it could, but at times would jump a piece that had gotten close. It also really valued moving up the board as much as possible to try and get a king. If these moves were available they were often chosen, which could be a residual impact of the `numpy.argmax` usage we discussed earlier. Finally, it's winning strategy after observing many games. It tries it's best to get as many kings as possible, and leaves it's back row completely full, thus blocking the opponent from making any kings. Once it has kings and black has no kings, the opponent will inevitably run out of moves, and the AI will win the game.

#### Further Possible Research
Some interesting thoughts we had along the way and after discussing was using a smart test subject to train against. When looking at the moves made, the AI will take risks because a random opponent will rarely select the right move that would punish that risk, such as moving up the board and being vulnerable for a jump. It has one thing in mind: getting a king. To try and prevent foolish behavior such as this, a smarter training opponent could be added that would punish the AI by jumping it when it could. 

Another interesting idea is to use a neural net to determine the reinforcements. Possible reinforcements could be jumping pieces, getting kings, favoring the sides as opposed to the middle, and not moving off the back wall. If weights were determined through a neaural net for these reinforcements, the AI could be much smarter, and produce more accurate reinforcements for certain moves. 

Finally, we thought that it would be interesting to represent the keys to the `Q` dictionary in different ways. One could include a localized board section, where selecting the locally optimum would hopefully result in the global optimum. Another could be recognizing similar board patterns and using those patterns with move types (normal, jump, king, etc) as inputs into the dictionary. The point of changing how keys are represented is to reduce the number of states, while still maintaining a decently accurate `Q` dictionary.

In [5]:
import io
from nbformat import current
import glob
nbfile = glob.glob('Reinforcement.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Reinforcement.ipynb is 2232
